In [21]:
%matplotlib widget
import numpy as np 
import copy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import functools as fn 
import glob, os
#from pathlib import Path
#Function that clears workspace variables
def reset_session():
    all_data = mRNA_lis1 = mRNA_lis2 = time_lis = []    
#Function to compute variance on lists of arrays
def variance(list_values, mean):
    def inner(n):
        r1 = ((n - mean)**2)/len(list_values)
        return r1
    results = list(map(inner, list_values))
    return results
#Function to compute fano factor and CV^2 on lists of arrays
def noise(list_variances, mean):
    def fano(variance):
        r1 = (variance / (mean))
        return r1
    def CVsq(variance):
        r2 = (variance / (mean)**2)
        return r2
    fano_factor = list(map(fano, list_variances))
    CV_sq = list(map(CVsq, list_variances))
    return fano_factor, CV_sq
#Function to compute list with crosscorrelations on lists of arrays
def correlator(list_mRNA1, list_mRNA2):
    result_list = []
    copy1 = copy.deepcopy(list_mRNA1); copy2 = copy.deepcopy(list_mRNA2)
    while (copy1 != []) and (len(copy1) == len(copy2)):
        array_one = copy1.pop(); array_two = copy2.pop()
        array_res = np.nan_to_num(sm.tsa.stattools.ccf(array_one, array_two))
        result_list.append(array_res)
    return result_list
#Read folder content and store file names
txt_folder = glob.glob(os.path.expanduser('~/Stochpy/*.txt'))
#txt_folder = Path('./').rglob('*.txt')

In [22]:
#Lists containing parsed data
length_data = len(txt_folder)
all_data = []; mRNA_lis1 = []; mRNA_lis2 = []; time_lis = []
mRNA_1 = []; mRNA_2  = []; time = []  
#Open all files and store content in lists
for i, name in enumerate(txt_folder):
    f = open(name, 'r')  
    all_data.append(f.readlines())
    f.close()
    text = all_data[i]
    for index, lines in enumerate(text[2:]): #Read all lines from stored files and split desired columns       
        num1 = float(lines.split("\t")[0])
        num2 = float(lines.split("\t")[4])
        num3 = float(lines.split("\t")[8])
        time.append(num1); mRNA_1.append(num2); mRNA_2.append(num3) #Append desired values into separated lists      
    mRNA_1 = np.array(mRNA_1); mRNA_2 = np.array(mRNA_2); time = np.array(time) #Converts lists into arrays
    mRNA_lis1.insert(i,mRNA_1); mRNA_lis2.insert(i,mRNA_2); time_lis.insert(i,time) #Insert arrays into lists 
    mRNA_1 = []; mRNA_2  = []; time = []     

In [24]:
#Create arrays with average of each data point in time series
avg_mRNA1 = np.array(fn.reduce((lambda a,b: (a+b)/2),mRNA_lis1))
avg_mRNA2 = np.array(fn.reduce((lambda a,b: (a+b)/2),mRNA_lis2))
avg_time = np.array(fn.reduce((lambda a,b: (a+b)/2),time_lis))
#Create arrays and lists of arrays with statistics on tracked species
avg_lis = [avg_mRNA1, avg_mRNA2]
avg_mRNAtot = np.array(fn.reduce((lambda a,b: (a+b)/2),avg_lis)) #Total average mRNA copy number over time [array]
cross_avg = np.array(fn.reduce(sm.tsa.stattools.ccf,avg_lis)) #Crosscorrelation of avg_mRNA1 and avg_mRNA2 copy number over time [array]
cross = np.array(fn.reduce((lambda a,b: (a+b)/2),correlator(mRNA_lis1, mRNA_lis2))) #Average crosscorrelation of arrays in mRNA_lis1 and mRNA_lis2 [array]
cross_avg_norm = cross_avg / max(cross_avg) #Normalized array of cross_avg [array]
cross_norm = cross / max(cross) #Normalized array of cross_avg [array]
auto1 = sm.tsa.stattools.acf(avg_mRNA1) #Autocorrelation of mRNA1 copy number over time [array]
auto2 = sm.tsa.stattools.acf(avg_mRNA2) #Autocorrelation of mRNA2 copy number over time [array]
mRNA1_var = variance(mRNA_lis1,avg_mRNA1) #Variance on every simulated trajectory for mRNA1 [list of arrays]
mRNA2_var = variance(mRNA_lis2,avg_mRNA2) #Variance on every simulated trajectory for mRNA2 [list of arrays]
fano_mRNA1, CVsq_mRNA1 = noise(mRNA1_var,avg_mRNA1) #Fano factor and CVsq for every simulated trajectory for mRNA1 [list of arrays]
fano_mRNA2, CVsq_mRNA2 = noise(mRNA2_var,avg_mRNA2) #Fano factor and CVsq for every simulated trajectory for mRNA2 [list of arrays]
avgfano_mRNA1 = np.array(fn.reduce((lambda a,b: (a+b)/2),fano_mRNA1)) #Average fano factor for mRNA1 [array]
avgfano_mRNA2 = np.array(fn.reduce((lambda a,b: (a+b)/2),fano_mRNA2)) #Average fano factor for mRNA2 [array]
avgCVsq_mRNA1 = np.array(fn.reduce((lambda a,b: (a+b)/2),CVsq_mRNA1)) #Average CVsq for mRNA1 [array]
avgCVsq_mRNA2 = np.array(fn.reduce((lambda a,b: (a+b)/2),CVsq_mRNA2)) #Average CVsq for mRNA2 [array]

/home/ffarina/myvenv/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1140: RuntimeWarning: invalid value encountered in true_divide
  return cvf / (np.std(x) * np.std(y))
/home/ffarina/myvenv/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:662: FutureWarning: The default number of lags is changing from 40 tomin(int(10 * np.log10(nobs)), nobs - 1) after 0.12is released. Set the number of lags to an integer to  silence this warning.
  FutureWarning,
/home/ffarina/myvenv/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:671: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning,
/home/ffarina/myvenv/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide
/home/ffarina/myvenv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


In [27]:
plt.figure()
plt.plot(cross_norm)
plt.xlabel("Time lag \u03C4")
plt.ylabel("Crosscorrelation")
plt.title("Average crosscorrelation for all trajectories (n=3000)")
plt.show

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<function matplotlib.pyplot.show(block=None)>

In [20]:
reset_session()